In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import napari
from PIL import Image

from scribbles_creator import *
from cellpose_data_handler import *

## Create predictions/segmentations on the cellpose dataset (with self-created scribbles)

In [ ]:
# folder_path = "./cellpose_results/data/run02_examples"
folder_path = "./cellpose_train_imgs"

mode = "all"
bins = [0.1]#, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
scribble_width = 2
all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
suff = all_suff[:1]
s = "a"

# Define the convpaint parameters
layer_list = [[0], [0,2], [0,2,5]] # layers to use for convpaint (depending on the model chosen)
scalings = [[1], [1,2], [1,2,4], [1,2,4,8]] # scalings for convpaint (downscaling the image)
model="vgg16" # 'vgg16', 'efficient_netb0', 'single_layer_vgg16', 'single_layer_vgg16_rgb', 'dino_vits16'
random_state = 123 # seed used for random forest classifier

# Predictions
for img_num in [1]:#range(0, 540, 10):#range(0, 10):
    for bin in [0.1]: #bins:
        for sc in [[1,2]]:#scalings:
            for l in [[0]]:#[[0,2,5]]:#layer_list:
                print(f"IMG {img_num}: bin {bin}, scalings {sc}, layers {l}")
                # get_cellpose_img_data(folder_path, img_num, load_img=True, load_gt=False, load_scribbles=False, mode=mode, bin=bin, suff=s, load_pred=False, pred_tag="convpaint")
                pred_cellpose_convpaint(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, save_res=True, show_res=True, layer_list=l, scalings=sc, model=model, random_state=random_state)
                pred_cellpose_ilastik(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, save_res=True, show_res=True)
                pred_cellpose_dino(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, save_res=True, show_res=True,
                                   dinov2_model='s', dinov2_layers=(), dinov2_scales=(), upscale_order=1, random_state=random_state)